In [30]:
import numpy as np
from timer_wrapper import clock

In [3]:
wheel = np.array([1, 7, 11, 13, 17, 19, 23, 29])

In [6]:
# Finding our offset values
offset_matrix = np.array([wheel*i for i in wheel]) % 30

We first take the "row" that we want to compute offset_values for (corresponding to a certain value in our wheel),
Then for each possible type of prime, call this `p_mod`(which is represented by our wheel) which corresponds to each row in the column matrix, 
Find the value which equals our "row" then get the corresponding wheel number of the column, call this $m$
Then compute the offset, by taking $m'$ (m' is just m but computed so that it is the same modulo wheel_size but larger than all our primes ignored) - `p_mod'` 

In [10]:
# we want to define an offset_dict = {prime: {row: offset}}
offset_dict = {}
for row in offset_matrix:
    p_mod = row[0]
    row_offset_dict = {}
    for m, row_value in zip(wheel, row):
        m_prime, p_mod_prime = m, p_mod
        if m <= 5: m_prime = m+30
        if p_mod <= 5: p_mod_prime = p_mod+30
        row_offset_dict[row_value] = m_prime - p_mod_prime #This is our offset
    offset_dict[p_mod] = row_offset_dict


{1: {1: 0, 7: -24, 11: -20, 13: -18, 17: -14, 19: -12, 23: -8, 29: -2},
 7: {7: 24, 19: 0, 17: 4, 1: 6, 29: 10, 13: 12, 11: 16, 23: 22},
 11: {11: 20, 17: -4, 1: 0, 23: 2, 7: 6, 29: 8, 13: 12, 19: 18},
 13: {13: 18, 1: -6, 23: -2, 19: 0, 11: 4, 7: 6, 29: 10, 17: 16},
 17: {17: 14, 29: -10, 7: -6, 11: -4, 19: 0, 23: 2, 1: 6, 13: 12},
 19: {19: 12, 13: -12, 29: -8, 7: -6, 23: -2, 1: 0, 17: 4, 11: 10},
 23: {23: 8, 11: -16, 13: -12, 29: -10, 1: -6, 17: -4, 19: 0, 7: 6},
 29: {29: 2, 23: -22, 19: -18, 17: -16, 13: -12, 11: -10, 7: -6, 1: 0}}

In [18]:
formula_dict = {} # {prime modulo: {row: (, ) -> coefficients}}
# the prime modulo in question must be in the indexable form
for prime_modulo in offset_dict:
    coeff_dict = {}
    b = prime_modulo - 30 if row_value > 1 else 1
    for row_value, offset in offset_dict[prime_modulo].items():
        a = row_value - 30 if row_value > 1 else 1
        coeff_dict[row_value] = (2*b + offset, (b*(b+offset) - a)//30)
    formula_dict[prime_modulo] = coeff_dict
formula_dict

{1: {1: (2, 0),
  7: (-22, 0),
  11: (-18, 0),
  13: (-16, 0),
  17: (-12, 0),
  19: (-10, 0),
  23: (-6, 0),
  29: (0, 0)},
 7: {7: (-22, 0),
  19: (-46, 18),
  17: (-42, 15),
  1: (-40, 13),
  29: (-36, 10),
  13: (-34, 9),
  11: (-30, 6),
  23: (-24, 1)},
 11: {11: (-18, 0),
  17: (-42, 15),
  1: (-38, 12),
  23: (-36, 11),
  7: (-32, 9),
  29: (-30, 7),
  13: (-26, 5),
  19: (-20, 1)},
 13: {13: (-16, 0),
  1: (-40, 13),
  23: (-36, 11),
  19: (-34, 10),
  11: (-30, 8),
  7: (-28, 7),
  29: (-24, 4),
  17: (-18, 1)},
 17: {17: (-12, 0),
  29: (-36, 10),
  7: (-32, 9),
  11: (-30, 8),
  19: (-26, 6),
  23: (-24, 5),
  1: (-20, 3),
  13: (-14, 1)},
 19: {19: (-10, 0),
  13: (-34, 9),
  29: (-30, 7),
  7: (-28, 7),
  23: (-24, 5),
  1: (-22, 4),
  17: (-18, 3),
  11: (-12, 1)},
 23: {23: (-6, 0),
  11: (-30, 6),
  13: (-26, 5),
  29: (-24, 4),
  1: (-20, 3),
  17: (-18, 3),
  19: (-14, 2),
  7: (-8, 1)},
 29: {29: (0, 0),
  23: (-24, 1),
  19: (-20, 1),
  17: (-18, 1),
  13: (-14, 1),

In [34]:
n=10**10
clock.start()
primesxmod30 = {prime_modulo: np.ones((n//30+1), dtype=bool) for prime_modulo in wheel}
wheel_ordered = [(7, -23), (11, -19), (13, -17), (17, -13), (19, -11), (23, -7), (29, -1), (1, 1)]
for i in range(1,int((n**0.5+1)/30)+1):
    for prime_modulo, a_prime in wheel_ordered:
        if primesxmod30[prime_modulo][i]:

            for row in primesxmod30:
                coeff1, coeff2= formula_dict[prime_modulo][row]
                primesxmod30[row][30*i*i + coeff1*i + coeff2::(30*i + a_prime)] = False
                # print(f"{prime_modulo}, row {row}" ,30*i*i + coeff1*i + coeff2,prime)
                # print(primesxmod30[row])
clock.stop()
primesxmod30

    

Elapsed time: 36.2717 seconds


{1: array([ True,  True,  True, ..., False, False, False]),
 7: array([ True,  True,  True, ..., False, False,  True]),
 11: array([ True,  True,  True, ..., False, False, False]),
 13: array([ True,  True,  True, ..., False,  True, False]),
 17: array([ True,  True,  True, ..., False, False, False]),
 19: array([ True,  True, False, ..., False, False, False]),
 23: array([ True,  True,  True, ..., False, False, False]),
 29: array([ True,  True,  True, ...,  True, False, False])}